<a href="https://colab.research.google.com/github/Ninaf-bot/neuralnetwork/blob/master/code_for_imbalanced_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import time
from tensorflow.keras.callbacks import TensorBoard
from imblearn.over_sampling import SMOTE
from sklearn import metrics
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample,class_weight
from sklearn.feature_selection import chi2


train_set=pd.read_csv('/content/drive/My Drive/dataSet/Train.csv')
train_set=train_set.drop(['Col1'],1)


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (746,835) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
fill_value=-999999

#train_set=train_set.fillna(fill_value)
train_set=train_set.convert_objects(convert_numeric=True)
train_set=train_set.fillna(fill_value)
x=train_set.drop(['Col2'],1)
y=train_set[['Col2']]
y_1=y[y['Col2']==1]
y_0=y[y['Col2']==0]
train_set.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  after removing the cwd from sys.path.


(17521, 2394)

In [0]:
y=np.array(y)
x_main=x
Scaler_0_1=preprocessing.MinMaxScaler(feature_range=(0,1000000)).fit(x)
x=Scaler_0_1.transform(x)
N_input=525
bestfeatures=SelectKBest(score_func=chi2,k=N_input)
select=bestfeatures.fit(x,y)
#dfscores = pd.DataFrame(select.scores_)
#dfcolumns = pd.DataFrame(x.columns)
#featureScores = pd.concat([dfcolumns,dfscores],axis=1)
#featureScores.columns = ['Specs','Score']
#featureScores=featureScores.nlargest(525,'Score')
x_select=select.transform(x)
x_select=pd.DataFrame(x_select)


In [0]:
x_select.columns=featureScores['Specs']
d=[]
for i in x.columns:
  if i not in x_select.columns:
    d.append(i)
x_not_select=pd.DataFrame()
for i in d:
  x_not_select[i]=x[i]


In [0]:

Scaler=preprocessing.StandardScaler().fit(x_select)
x_select_scaled=Scaler.transform(x_select)

x_train,x_test,y_train,y_test=train_test_split(x_select_scaled,y,test_size=0.09,shuffle=1,random_state=0)

In [6]:
sm=SMOTE(random_state=27,ratio=1)
x_train_sm,y_train_sm=sm.fit_sample(x_train,y_train)
x_train_sm.max().max()

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


118.57778986052486

In [0]:
x_train=pd.DataFrame(x_train)
y_train=pd.DataFrame(y_train)
dataset=pd.concat([x_train,y_train],axis=1)
dataset_1=dataset[dataset.iloc[:,-1]==1]
dataset_0=dataset[dataset.iloc[:,-1]==0]

upsample=resample(dataset_1,replace=1,n_samples=len(dataset_0),random_state=27)

dataset_1_usm=pd.concat([dataset_1,upsample])
dataset_train=pd.concat([dataset_0,dataset_1_usm])
x_train_usm=dataset_train.iloc[:,0:-1]
y_train_usm=dataset_train.iloc[:,-1]

In [7]:

#Scaler=preprocessing.RobustScaler().fit(x_select)
#Scaler=preprocessing.Normalizer().fit(x_select)
#Scaler=preprocessing.MinMaxScaler(feature_range=(-1,1)).fit(x_select)
#Scaler=preprocessing.MaxAbsScaler().fit(x_select)



#x_train_local_scaled=Scaler.transform(x_train_local)
#x_test_local_scaled=Scaler.transform(x_test_local)
train=pd.DataFrame(x_train_sm)
print(x_train.max().max())
print(x_train.min().min())
x_train_sm=x_train_sm/119
x_train=x_train/119
x_test=x_test/119
train=pd.DataFrame(x_train_sm)
print(x_train.max().max())
print(x_train.min().min())

#train.to_csv('n.csv')

118.57778986052486
-2.022462470615713
0.9964520156346627
-0.01699548294635053


In [8]:
y_train=pd.DataFrame(y_train)
class_weights = class_weight.compute_class_weight('balanced',(0,1),y_train)
y_train=np.array(y_train)
class_weights

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0.55534657, 5.01699182])

In [98]:



#name='model3-{}'.format(int(time.time()))
#tensorboard=TensorBoard(log_dir='logs/{}'.format(name))



model=tf.keras.Sequential([
                     layers.Dense(600,activation='relu',input_shape=(N_input,)),
                     layers.Dropout(0.2),
                     layers.Dense(600,activation='relu'),
                     layers.Dropout(0.2),
                     #layers.Dense(500,activation='relu'),
                     #layers.Dropout(0.2),
                     #layers.Dense(500,activation='relu'),
                     layers.Dense(2,activation='softmax')
])
#opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
opt = tf.keras.optimizers.Adam()
model.compile(optimizer=opt,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

checkpointer=ModelCheckpoint(filepath='model1.hdf5',monitor='val_loss',verbose=2,save_best_only=True)

model.fit(x_train,y_train,epochs=10,batch_size=200,validation_data=(x_test,y_test),callbacks=[checkpointer],class_weight={0:1.6,1:5.01699182})
#model.evaluate(x_test_local_scaled,y_test_local)

new_model=load_model('model1.hdf5')

y_pred=new_model.predict(x_test)

y_pred=pd.DataFrame(y_pred)
b=[]
for i in y_pred.values:
  if i[0]>i[1]:
    b.append(0)
  else:
    b.append(1)

y_pred=pd.DataFrame(b)

metrics.accuracy_score(y_test,y_pred),metrics.f1_score(y_test,y_pred),metrics.precision_score(y_test,y_pred),metrics.recall_score(y_test,y_pred)

Train on 15944 samples, validate on 1577 samples
Epoch 1/10
15800/15944 [============================>.] - ETA: 0s - loss: 1.0543 - acc: 0.8837
Epoch 00001: val_loss improved from inf to 0.36593, saving model to model1.hdf5
15944/15944 [==============================] - 13s 836us/sample - loss: 1.0529 - acc: 0.8841 - val_loss: 0.3659 - val_acc: 0.8878
Epoch 2/10
15800/15944 [============================>.] - ETA: 0s - loss: 0.9966 - acc: 0.8821
Epoch 00002: val_loss did not improve from 0.36593
15944/15944 [==============================] - 3s 162us/sample - loss: 0.9972 - acc: 0.8820 - val_loss: 0.3689 - val_acc: 0.8814
Epoch 3/10
15800/15944 [============================>.] - ETA: 0s - loss: 0.9906 - acc: 0.8778
Epoch 00003: val_loss did not improve from 0.36593
15944/15944 [==============================] - 3s 161us/sample - loss: 0.9879 - acc: 0.8782 - val_loss: 0.3878 - val_acc: 0.8802
Epoch 4/10
15800/15944 [============================>.] - ETA: 0s - loss: 0.9771 - acc: 0.8810
E

(0.8826886493341788,
 0.3321299638989169,
 0.4380952380952381,
 0.26744186046511625)

In [106]:
new_model=load_model('model1.hdf5')

y_pred=new_model.predict(x_test)

y_pred=pd.DataFrame(y_pred)
b=[]
for i in y_pred.values:
  if i[0]>0.5:
    b.append(0)
  else:
    b.append(1)

y_pred=pd.DataFrame(b)

metrics.accuracy_score(y_test,y_pred),metrics.f1_score(y_test,y_pred),metrics.precision_score(y_test,y_pred),metrics.recall_score(y_test,y_pred)

(0.8826886493341788,
 0.3321299638989169,
 0.4380952380952381,
 0.26744186046511625)

In [0]:
model=tf.keras.Sequential([
                     layers.Dense(500,activation='relu',input_shape=(525,)),
                     layers.Dropout(0.2),
                     layers.Dense(500,activation='relu'),
                     layers.Dropout(0.2),
                     layers.Dense(500,activation='relu'),
                     #layers.Dropout(0.2),
                     #layers.Dense(100,activation='relu'),
                     layers.Dense(2,activation='softmax')
])
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
#opt = tf.keras.optimizers.Adam()
model.compile(optimizer=opt,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

checkpointer=ModelCheckpoint(filepath='model2.hdf5',monitor='val_loss',verbose=2,save_best_only=True)

model.fit(x_train_sm,y_train_sm,epochs=10,batch_size=200,validation_data=(x_test,y_test),callbacks=[checkpointer])

new_model=load_model('model2.hdf5')

y_pred=new_model.predict(x_test)

y_pred=pd.DataFrame(y_pred)
b=[]
for i in y_pred.values:
  if i[0]>i[1]:
    b.append(0)
  else:
    b.append(1)

y_pred=pd.DataFrame(b)

metrics.accuracy_score(y_test,y_pred),metrics.f1_score(y_test,y_pred),metrics.precision_score(y_test,y_pred),metrics.recall_score(y_test,y_pred)

Train on 17922 samples, validate on 1577 samples
Epoch 1/10
17800/17922 [============================>.] - ETA: 0s - loss: 0.4681 - acc: 0.7996
Epoch 00001: val_loss improved from inf to 0.31670, saving model to model2.hdf5
17922/17922 [==============================] - 22s 1ms/sample - loss: 0.4680 - acc: 0.7996 - val_loss: 0.3167 - val_acc: 0.9017
Epoch 2/10
16800/17922 [===========================>..] - ETA: 0s - loss: 0.4424 - acc: 0.8082
Epoch 00002: val_loss did not improve from 0.31670
17922/17922 [==============================] - 1s 41us/sample - loss: 0.4429 - acc: 0.8078 - val_loss: 0.3368 - val_acc: 0.8890
Epoch 3/10
17000/17922 [===========================>..] - ETA: 0s - loss: 0.4354 - acc: 0.8099
Epoch 00003: val_loss did not improve from 0.31670
17922/17922 [==============================] - 1s 40us/sample - loss: 0.4348 - acc: 0.8100 - val_loss: 0.3340 - val_acc: 0.8960
Epoch 4/10
16600/17922 [==========================>...] - ETA: 0s - loss: 0.4268 - acc: 0.8148
Epoch

(0.901712111604312, 0.037267080745341616, 0.5, 0.01935483870967742)

In [0]:
model=tf.keras.Sequential([
                     layers.Dense(500,activation='relu',input_shape=(525,)),
                     layers.Dropout(0.2),
                     layers.Dense(500,activation='relu'),
                     layers.Dropout(0.2),
                     layers.Dense(500,activation='relu'),
                     #layers.Dropout(0.2),
                     #layers.Dense(100,activation='relu'),
                     layers.Dense(2,activation='softmax')
])
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
#opt = tf.keras.optimizers.Adam()
model.compile(optimizer=opt,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

checkpointer=ModelCheckpoint(filepath='model3.hdf5',monitor='val_loss',verbose=2,save_best_only=True)

model.fit(x_train,y_train,epochs=20,batch_size=200,validation_data=(x_test,y_test),callbacks=[checkpointer])

new_model=load_model('model3.hdf5')

y_pred=new_model.predict(x_test)

y_pred=pd.DataFrame(y_pred)
b=[]
for i in y_pred.values:
  if i[0]>i[1]:
    b.append(0)
  else:
    b.append(1)

y_pred=pd.DataFrame(b)

metrics.accuracy_score(y_test,y_pred),metrics.f1_score(y_test,y_pred),metrics.precision_score(y_test,y_pred),metrics.recall_score(y_test,y_pred)

In [0]:
clf=RandomForestClassifier(n_estimators=200)
clf.fit(x_train_sm,y_train_sm)

y_pred=clf.predict(x_test)

metrics.accuracy_score(y_test,y_pred),metrics.f1_score(y_test,y_pred),metrics.precision_score(y_test,y_pred),metrics.recall_score(y_test,y_pred)

(0.8490805326569436,
 0.14388489208633093,
 0.1834862385321101,
 0.11834319526627218)

In [107]:
predict_set=pd.read_csv('/content/drive/My Drive/dataSet/Test.csv')
x_predict=predict_set.drop(['Col1'],1)
ids=predict_set['Col1']

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (700,731,740,752,761,789,811,820,829,841,850) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [108]:
#x_predict=x_predict.fillna(fill_value)
x_predict=x_predict.convert_objects(convert_numeric=True)
x_predict=x_predict.fillna(fill_value)
x_predict.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


(20442, 2393)

In [0]:
x_predict_0_1=Scaler_0_1.transform(x_predict)

x_predict_select=select.transform(x_predict_0_1)
#x_predict_select=pd.DataFrame(x_predict_select)


In [40]:
x_predict_select.columns=featureScores['Specs']

f=[]
for i in x_predict.columns:
  if i not in x_predict_select.columns:
    f.append(i)
x_predict_not_select=pd.DataFrame()
for i in f:
  x_predict_not_select[i]=x[i]

#x_predict_not_select_scaled=pca_scaler.transform(x_predict_not_select)

#x_predict_not_select_pca=pca.transform(x_predict_not_select)
#x_predict_not_select_pca=pd.DataFrame(x_predict_not_select_pca)
#x_predict_new=pd.concat([x_predict_select,x_predict_not_select_pca],axis=1)

NameError: ignored

In [0]:
x_predict_scaled=Scaler.transform(x_predict_select)
x_predict_scaled=x_predict_scaled/119

new_model=load_model('model1.hdf5')

a=new_model.predict(x_predict_scaled)

In [0]:
a=pd.DataFrame(a)
b=[]
for i in a.values:
  if i[0]>0.5:
    b.append(0)
  else:
    b.append(1)


In [112]:
predicted=pd.DataFrame({'Col1':ids,'Col2':b})


predicted.to_csv('predicted_select_neural.csv',index=False)
model.save('model_neural.model')
predicted

,Col1,Col2
0,RIGD58ZWD,0
1,RIH660YDS,0
2,RIH660Q96,0
3,RIYDO15W1,0
4,RIYBGC1ZD,0
5,RIYBQ5SWR,0
6,RIYBQ5SD5,0
7,TI5HU5XY,0
8,TIYHGOSE,0
9,T90K61QJ,0
